In [1]:
import sys
sys.path.insert(0, '.')
import swarmist as sw

## PSO

In [3]:
pso_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM C1 = 2.05 BOUNDED BY (0, 8)
    PARAM C2 = 2.05 BOUNDED BY (0, 8) 
    PARAM CHI = 0.7298 BOUNDED BY (0, 1)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            VELOCITY= PARAM(CHI) * (
                DELTA 
                + PARAM(C1) * RANDOM() * (BEST-POS)
                + PARAM(C2) * RANDOM() * (SWARM_BEST()-POS)
            )
            POS = POS + VELOCITY
        ) 
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## BB

In [4]:
bb_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            MU= (SWARM_BEST()+BEST)/2
            SD = ABS(SWARM_BEST()-BEST)
            POS = RANDOM_NORMAL(LOC=MU, SCALE=SD)
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## FIPS

In [5]:
fips_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM PHI = 4.1 BOUNDED BY (0, 8)
    PARAM CHI = 0.7298 BOUNDED BY (0, 1)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM() WITH TOPOLOGY LBEST SIZE(2)
    SELECT ALL (
        UPDATE (
            NEIGHBORS = NEIGHBORHOOD()
            N = COUNT(NEIGHBORS)
            W = RANDOM(SIZE=N)
            PHI = SUM(W) * (PARAM(PHI) / N)
            PM = AVG(NEIGHBORS, W)
            SCT = PHI * (PM - POS)
            POS = POS + PARAM(CHI) * (DELTA + SCT)
        ) 
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## DE (Binomial)

In [6]:
de_bin_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM F = 0.5 BOUNDED BY (0, 1)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        USING BINOMIAL RECOMBINATION WITH PROBABILITY 0.6
        UPDATE (
            POS = PICK_RANDOM(UNIQUE) + PARAM(F) * (PICK_RANDOM(UNIQUE) - PICK_RANDOM(UNIQUE)) 
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## DE (Exponential)

In [7]:
de_exp_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM F = 0.5 BOUNDED BY (0, 1)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        USING EXPONENTIAL RECOMBINATION WITH PROBABILITY 0.6
        UPDATE (
            POS = PICK_RANDOM(UNIQUE) + PARAM(F) * (PICK_RANDOM(UNIQUE) - PICK_RANDOM(UNIQUE)) 
        ) WHEN IMPROVED = TRUE 
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## DE RAND-TO-BEST

In [8]:
de_r2b_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM F = 0.5 BOUNDED BY (0, 1)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        USING BINOMIAL RECOMBINATION WITH PROBABILITY 0.6
        UPDATE (
            POS = RAND_TO_BEST(WITH PROBABILITY 0.5) + PARAM(F) * (PICK_RANDOM(UNIQUE) - PICK_RANDOM(UNIQUE)) 
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## DE CURRENT-TO-BEST

In [10]:
de_r2b_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM F = 0.5 BOUNDED BY (0, 1)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        USING BINOMIAL RECOMBINATION WITH PROBABILITY 0.6
        UPDATE (
            POS = CURRENT_TO_BEST(WITH PROBABILITY 0.5) + PARAM(F) * (PICK_RANDOM(UNIQUE) - PICK_RANDOM(UNIQUE)) 
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## JAYA

In [11]:
jaya_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            ABS_POS = ABS(POS)
            POS = POS + RANDOM() * (SWARM_BEST() - ABS_POS) - RANDOM() * (SWARM_WORST() - ABS_POS)
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## ABC

In [12]:
abc_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        USING RANDOM RECOMBINATION SIZE(1)
        UPDATE (
            POS = RANDOM_UNIFORM(LOW=-1, HIGH=1) * (POS - PICK_RANDOM())
        ) WHEN IMPROVED = TRUE
    )
    SELECT SIZE(1) WHERE TRIALS > POPULATION_SIZE*NDIMS ORDER BY TRIALS DESC (
        INIT RANDOM_UNIFORM()
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## CS

In [14]:
cs_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM PA = 0.25 BOUNDED BY (0, 1)
    PARAM ALPHA = 1 BOUNDED BY (0, 1)
    PARAM MU = 1.5 BOUNDED BY (0,3)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            POS = POS + PARAM(ALPHA) * RANDOM_LEVY(LOC=PARAM(MU)) * (POS - SWARM_BEST())
        ) WHEN IMPROVED = TRUE
    )
    SELECT ALL (
        USING RECOMBINATION WITH PROBABILITY PARAM(PA)
        UPDATE (
            POS = POS + RANDOM() * (PICK_RANDOM(UNIQUE) - PICK_RANDOM(UNIQUE))
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## FF

In [15]:
ff_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X ** 2)
)
USING (
    PARAM ALPHA = 1 BOUNDED BY (0, 2)
    PARAM DELTA = 0.97 BOUNDED BY (0, 1)
    PARAM BETA = 1 BOUNDED BY (0, 2)
    PARAM GAMMA = 0.01 BOUNDED BY (0, 1)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            ALPHA = ( PARAM(ALPHA) * PARAM(DELTA) ) ** CURR_GEN
            VALUES = MAP(ALL(), (REF) => IF_THEN(REF.FIT < FIT, REF.POS, 0))
            POS = REDUCE(
                VALUES, 
                (ACC, VAL) => ACC + (
                    ( PARAM(BETA) * EXP( -1 * PARAM(GAMMA) * (VAL - ACC)**2 ))
                    * (VAL - ACC) + ALPHA * RANDOM_UNIFORM(LOW=-1, HIGH=1)
                ),
                POS
            )
        )
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## GWO

In [2]:
gwo_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM A = 2 BOUNDED BY (0, 10)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            A = PARAM(A) - CURR_GEN * ( PARAM(A) / MAX_GEN ) 
            POS = AVG(
                REDUCE(
                    SWARM_BEST(3), 
                    (ACC, REF) => ACC + ( A * ABS( (2 * RANDOM()) * REF.POS - POS) ),
                    0
                )
            )
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## SCA

In [3]:
sca_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM A = 2 BOUNDED BY (0, 10)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            A = PARAM(A) - CURR_GEN * ( PARAM(A) / MAX_GEN )
            SC = REPEAT(
                IF_THEN(
                    RANDOM(SIZE=1) < 0.5, 
                    SIN( RANDOM_UNIFORM(LOW=0, HIGH=2*PI(), SIZE=1) ),
                    COS( RANDOM_UNIFORM(LOW=0, HIGH=2*PI(), SIZE=1) )
                ), 
                NDIMS) 
            POS = POS + ( A * SC * ABS( RANDOM() * SWARM_BEST() - POS ) ) 
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## TLBO

In [4]:
tlbo_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM A = 2 BOUNDED BY (0, 10)
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            TF = IF_THEN(RANDOM(SIZE=1) < 0.5, 1, 2)
            POS = POS + ( RANDOM() * ( SWARM_BEST() - TF*AVG(ALL()) ) ) 
        ) WHEN IMPROVED = TRUE
    )
    SELECT ALL (
        UPDATE (
            POS = REDUCE(
                PICK_RANDOM(UNIQUE),
                (ACC, REF) => ACC + IF_THEN(
                    REF.FIT < FIT,
                    RANDOM() * (REF.POS - POS),
                    RANDOM() * (POS - REF.POS)
                ),
                POS
            )
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

## WO

In [5]:
wo_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAM A = 2 BOUNDED BY (0, 10)
    PARAM BETA = .5 BOUNDED BY (0,1) 
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            a = PARAM(A) - CURR_GEN * ( PARAM(A) / MAX_GEN )
            A = 2 * ( a * RANDOM() ) - a
            ATTACK = IF_THEN(RANDOM(SIZE=1) <.5, TRUE, FALSE)
            REF = IF_THEN( ATTACK = TRUE AND NORM(A) < 1, PICK_RANDOM(UNIQUE), SWARM_BEST() )
            L = IF_THEN(ATTACK = TRUE,  RANDOM_UNIFORM(LOW=-1, HIGH=1), 0)
            D = IF_THEN(
                ATTACK = TRUE, 
                ABS( REF - POS ), 
                ABS( (2 * RANDOM()) * REF - POS )
            )
            POS = IF_THEN(
                ATTACK = TRUE,
                D * EXP( PARAM(BETA) * L ) * COS( 2 * PI() * L ) + REF,
                REF - A * D
            )
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")